# Q1: Pairwise Sequence Alignment

This workbook will illustrate a simple implementation of Needleman-Wunsch global pairwise alignment. The approach presented won't be the most efficient way possible in Python (for loops!), but it implements the pseudocode we discussed in class. 

See [Python for Bioinformatics](https://catalog.libraries.psu.edu/catalog/19170896) for more details. 

First, let's define the scoring schema. We need a similarity matrix for match and mismatch scores, and we need a gap penalty. We will also define indices for the valid base characters (so we can use them in the scoring matrix later). We will assume a DNA alphabet throughout. 

In [3]:
from numpy import *

In [ ]:
## Indices for each base
index = {
    'A': 0, 
    'C': 1, 
    'G': 2, 
    'T': 3}

## Set the gap penalty
gap = -2

## Match/mismatch similarity matrix (defined as numpy array).
matchscore = 3
mismatchscore=-1
sim = full((4,4), mismatchscore)
fill_diagonal(sim, matchscore)

sim

array([[ 3, -1, -1, -1],
       [-1,  3, -1, -1],
       [-1, -1,  3, -1],
       [-1, -1, -1,  3]])

Now let's make a function that implements Needleman-Wunsch, returning the score matrix and the arrow matrix

In [ ]:
#arguments: similarity matrix, gap penalty, sequence1, sequence2
def NeedlemanWunsch(sim_mat, gap_pen, seq1, seq2):
    l1, l2 = len(seq1), len(seq2)
    scores = zeros( (l1+1, l2+1), int )
    arrows = zeros( (l1+1, l2+1), int )

    #Define the scores for the first row and first column
    scores[0,:] = arange(l2+1) * gap_pen
    scores[:,0] = arange(l1+1) * gap_pen
    arrows[0,:] = ones(l2+1)
    arrows[:,0] = zeros(l1+1)

    #Dynamic programming part - iterate over rows & columns
    for i in range(1, l1+1):
        for j in range(1, l2+1):
            f=zeros(3) #f is going to store our three scoring options in the current cell
            
            f[0] = scores[i-1, j]+gap_pen  #Vertical gap option
            f[1] = scores[i, j-1]+gap_pen  #Horizontal gap option
            n1 = index[seq1[i-1]]
            n2 = index[seq2[j-1]]
            f[2] = scores[i-1, j-1] + sim_mat[n1, n2] #Diagonal match/mismatch option

            scores[i,j]= f.max()
            arrows[i,j] = f.argmax();

    return scores, arrows

Let's test the above function on a pair of sequences.

In [ ]:
seq1 = "ACCGATGTACTGTAGGT"
seq2 = "GAGTCTACTGTTTAATC"

s, a = NeedlemanWunsch(sim, gap, seq1, seq2)

print(s)
print(a)

[[  0  -2  -4  -6  -8 -10 -12 -14 -16 -18 -20 -22 -24 -26 -28 -30 -32 -34]
 [ -2  -1   1  -1  -3  -5  -7  -9 -11 -13 -15 -17 -19 -21 -23 -25 -27 -29]
 [ -4  -3  -1   0  -2   0  -2  -4  -6  -8 -10 -12 -14 -16 -18 -20 -22 -24]
 [ -6  -5  -3  -2  -1   1  -1  -3  -1  -3  -5  -7  -9 -11 -13 -15 -17 -19]
 [ -8  -3  -5   0  -2  -1   0  -2  -3  -2   0  -2  -4  -6  -8 -10 -12 -14]
 [-10  -5   0  -2  -1  -3  -2   3   1  -1  -2  -1  -3  -5  -3  -5  -7  -9]
 [-12  -7  -2  -1   1  -1   0   1   2   4   2   1   2   0  -2  -4  -2  -4]
 [-14  -9  -4   1  -1   0  -2  -1   0   2   7   5   3   1  -1  -3  -4  -3]
 [-16 -11  -6  -1   4   2   3   1  -1   3   5  10   8   6   4   2   0  -2]
 [-18 -13  -8  -3   2   3   1   6   4   2   3   8   9   7   9   7   5   3]
 [-20 -15 -10  -5   0   5   3   4   9   7   5   6   7   8   7   8   6   8]
 [-22 -17 -12  -7  -2   3   8   6   7  12  10   8   9  10   8   6  11   9]
 [-24 -19 -14  -9  -4   1   6   7   5  10  15  13  11   9   9   7   9  10]
 [-26 -21 -16 -11  -6  -1

As you can see, the score matrix contains all scores, culminating in the overall alignment score in the bottom right corner. The arrow matrix contains the choices taken to generate the score in each cell. 0 for vertical gap, 1 for horizontal gap, 2 for diagonal. 

Next, we need to generate the actual alignment by performing a traceback operation from the bottom right corner to the top left. 

In [ ]:
#arguments: arrow matrix, sequence1, sequence2
def Backtrace(arrows, seq1, seq2):
    #st1 and st2 will store the aligned sequence strings
    st1, st2 = '',''

    # v and h will be indices that start at the bottom right position
    v = arrows.shape[0]-1
    h = arrows.shape[1]-1

    while v>0 or h>0 :
        if arrows[v,h] == 0:   #Vertical gap (insert gap in seq2)
            st1 += seq1[v-1]
            st2 += '-'
            v-=1
        elif arrows[v,h] == 1: #Horizontal gap (insert gap in seq1)
            st1 += '-'
            st2 += seq2[h-1]
            h-=1
        elif arrows[v,h] == 2: #Diagonal transition (aligned bases)
            st1 += seq1[v-1]
            st2 += seq2[h-1]
            v-=1
            h-=1
    
    #st1 and st2 now have the aligned sequences in reverse order
    st1 = st1[::-1]
    st2 = st2[::-1]

    return st1, st2

Let's try it on the arrow matrix that was computed above:

In [ ]:
seq1 = "ACCGATGTACTGTAGGT"
seq2 = "GAGTCTACTGTTTAATC"

s1, s2 = Backtrace(a, seq1, seq2)

print(s1)
print(s2)

-ACCGATGTACTGT--AGGT-
GA--G-TCTACTGTTTAA-TC


Finally, put it all together for longer sequences

In [ ]:
seq1 = "ACCGATGTACTGTAGGT"
seq2 = "GAGTCTACTGTTTAATC"



s, a = NeedlemanWunsch(sim, gap, seq1, seq2)
ascore = s[len(seq1), len(seq2)]

s1, s2 = Backtrace(a, seq1, seq2)

print("Alignment score = ", ascore)
print(s1)
print(s2)

Alignment score =  15
-ACCGATGTACTGT--AGGT-
GA--G-TCTACTGTTTAA-TC


We could also plot the scoring matrix as a heatmap, just to get a sense of the alignment scoring landscape. 

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt
ax = sns.heatmap(s)
plt.show()

## Smith-Waterman local alignment

In [ ]:
#arguments: similarity matrix, gap penalty, sequence1, sequence2
def SmithWaterman(sim_mat, gap_pen, seq1, seq2):
    l1, l2 = len(seq1), len(seq2)
    scores = zeros( (l1+1, l2+1), int )
    arrows = zeros( (l1+1, l2+1), int )

    #Define the scores for the first row and first column
    scores[0,:] = zeros(l2+1)
    scores[:,0] = zeros(l1+1)
    arrows[0,:] = ones(l2+1)*3
    arrows[:,0] = ones(l1+1)*3

    maxscore = 0
    maxi=0
    maxj=0

    #Dynamic programming part - iterate over rows & columns
    for i in range(1, l1+1):
        for j in range(1, l2+1):
            f=zeros(4) #f is going to store our three scoring options in the current cell
            
            f[0] = scores[i-1, j]+gap_pen  #Vertical gap option
            f[1] = scores[i, j-1]+gap_pen  #Horizontal gap option
            n1 = index[seq1[i-1]]
            n2 = index[seq2[j-1]]
            f[2] = scores[i-1, j-1] + sim_mat[n1, n2] #Diagonal match/mismatch option
            f[3]=0

            scores[i,j]= f.max()
            arrows[i,j] = f.argmax();

            if(scores[i,j]>maxscore):
              maxscore=scores[i,j]
              maxi=i
              maxj=j

    return scores, arrows, maxi, maxj

#arguments: arrow matrix, sequence1, sequence2
def SWBacktrace(arrows, seq1, seq2, maxi, maxj):
    #st1 and st2 will store the aligned sequence strings
    st1, st2 = '',''

    # v and h will be indices that start at the bottom right position
    v = maxi
    h = maxj

    while v>0 or h>0 :
        if arrows[v,h] == 0:   #Vertical gap (insert gap in seq2)
            st1 += seq1[v-1]
            st2 += '-'
            v-=1
        elif arrows[v,h] == 1: #Horizontal gap (insert gap in seq1)
            st1 += '-'
            st2 += seq2[h-1]
            h-=1
        elif arrows[v,h] == 2: #Diagonal transition (aligned bases)
            st1 += seq1[v-1]
            st2 += seq2[h-1]
            v-=1
            h-=1
        elif arrows[v,h] == 3: #STOP
          v=0
          h=0
    
    #st1 and st2 now have the aligned sequences in reverse order
    st1 = st1[::-1]
    st2 = st2[::-1]

    return st1, st2

In [ ]:
seq1 = "ACCGATGTACTGTAGGT"
seq2 = "GAGTCTACTGTTTAATC"



s, a,mi,mj = SmithWaterman(sim, gap, seq1, seq2)
ascore = s[mi, mj]

s1, s2 = SWBacktrace(a, seq1, seq2,mi,mj)

print("Alignment score = ", ascore)
print(s1)
print(s2)
print(mi,mj)
print(s)
print(a)

Alignment score =  24
GA-TGTACTGT
GAGTCTACTGT
13 11
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  1  0  0  0  3  1  0  0  0  0  0  3  3  1  0]
 [ 0  0  1  2  0  3  1  1  6  4  2  0  0  0  1  2  2  4]
 [ 0  0  0  0  1  3  2  0  4  5  3  1  0  0  0  0  1  5]
 [ 0  3  1  3  1  1  2  1  2  3  8  6  4  2  0  0  0  3]
 [ 0  1  6  4  2  0  0  5  3  1  6  7  5  3  5  3  1  1]
 [ 0  0  4  5  7  5  3  3  4  6  4  9 10  8  6  4  6  4]
 [ 0  3  2  7  5  6  4  2  2  4  9  7  8  9  7  5  4  5]
 [ 0  1  2  5 10  8  9  7  5  5  7 12 10 11  9  7  8  6]
 [ 0  0  4  3  8  9  7 12 10  8  6 10 11  9 14 12 10  8]
 [ 0  0  2  3  6 11  9 10 15 13 11  9  9 10 12 13 11 13]
 [ 0  0  0  1  6  9 14 12 13 18 16 14 12 12 10 11 16 14]
 [ 0  3  1  3  4  7 12 13 11 16 21 19 17 15 13 11 14 15]
 [ 0  1  2  1  6  5 10 11 12 14 19 24 22 20 18 16 14 13]
 [ 0  0  4  2  4  5  8 13 11 12 17 22 23 21 23 21 19 17]
 [ 0  3  2  7  5  3  6 11 12 10 15 20 21 22 21 22 20 18]
 [ 0  3  2  5  6  4  4  9 10 11 13 1

# Question 3: Burrows-Wheeler Transform

## Part 1: Make the BWT

In [4]:
from numpy import *

In [5]:
def makeBWT(seq):

  #Add end-of-sequence character
  seq = seq + "$"

  #Make rotations
  rotations = []
  rotations.append(seq)
  for i in range(1,len(seq)):
    seq = seq[-1:] + seq[:-1]
    rotations.append(seq)

  #Sort 
  sorted_rotations = sort(rotations).tolist()

  #BWT is the last column
  bwt = ''.join([x[-1] for x in sorted_rotations])
  return(bwt)

In [6]:
#Test
seq = "GATCGATCAT"
bwt = makeBWT(seq)
print(bwt)

TCGGTTC$AAA


## Part 2: Reverse the BWT

In [41]:
def reverseBWT(bwt):

    #Count the letters used in the BWT. 
    letter_counts = {}
    for l in bwt: 
        if(l in letter_counts):
            letter_counts[l]+=1
        else:
            letter_counts[l]=1
    uniq_letters = sorted(letter_counts.keys())

    #Make the L-F mapping by iterating through the characters in the BWT.
    #Because the first column is always an alphabetical sort of the BWT, the L-F
    # mapping will always have $ = 0, first A = 1, last A = #numAs, first C = #numAs +1, etc.
    lf_map = zeros(len(bwt), dtype=int)
    curr_count = 0
    for c in uniq_letters:
        for i in range(len(bwt)):
            if(bwt[i] == c):
                lf_map[i]=curr_count
                curr_count+=1
    print("L-F map: ", lf_map)

    #Use the L-F map to reverse the BWT. 
    #Start at index zero, because that is where the $ index will always point to
    # (i.e., the character in bwt[0] is always the last character in the genome)
    seq=''
    curr_index=0;
    for i in range(len(bwt)-1):
        seq = bwt[curr_index]+seq
        print("Step: ",curr_index," --> ", lf_map[curr_index])
        curr_index = lf_map[curr_index]

    return seq

In [42]:
#Test
bwt = "TCGGTTC$AAA"
seq = reverseBWT(bwt)
print("Genome = ",seq)

L-F map:  [ 8  4  6  7  9 10  5  0  1  2  3]
Step:  0  -->  8
Step:  8  -->  1
Step:  1  -->  4
Step:  4  -->  9
Step:  9  -->  2
Step:  2  -->  6
Step:  6  -->  5
Step:  5  -->  10
Step:  10  -->  3
Step:  3  -->  7
Genome =  GATCGATCAT
